# Introducción a las Redes Neuronales INF-395 II-2025


* Nombre: Alessandro Bruno Cintolesi Rodríguez
* ROL: 202173541-0

In [ ]:
# Import torch
import torch

# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

# Setup random seed
RANDOM_SEED = 42

## 1. Cree un conjunto de datos de clasificación binaria con Scikit-Learn [`make_moons()`](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_moons.html) function.
* Para mantener la coherencia, el conjunto de datos debe tener 1000 muestras y un `random_state=42`.
* Convierte los datos en tensores de PyTorch.
* Divida los datos en conjuntos de entrenamiento y prueba utilizando `train_test_split` con 80 % de entrenamiento y 20 % de prueba.

In [ ]:
from sklearn.datasets import make_moons

n_samples=1000
X, y = make_moons(n_samples, random_state=RANDOM_SEED)

In [ ]:
import pandas as pd

moons = pd.DataFrame({"X1": X[:, 0], "X2": X[:, 1], "label": y })
moons.head(10)

In [ ]:
import matplotlib.pyplot as plt

moons['X1'].plot(kind='hist', bins=20, title='X1')
plt.gca().spines[['top', 'right',]].set_visible(False)

In [ ]:
moons.label.value_counts()

In [ ]:
plt.scatter(x=X[:, 0], y=X[:, 1],  c=y, cmap=plt.cm.RdYlBu);

In [ ]:
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.float)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
len(X_train), len(X_test), len(y_train), len(y_test)

## 2. Construya un modelo subclasificando `nn.Module` que incorpore funciones de activación no lineal y sea capaz de ajustar los datos que creó en 1.
* Siéntase libre de utilizar cualquier combinación de capas (lineales y no lineales) que desees.

In [ ]:
import torch
from torch import nn

class MoonModelV0(nn.Module):
	## Your code here ##
	def __init__(self, in_features=2, hidden=32, out_features=1):
		super().__init__()
		self.layer_1 = nn.Linear(in_features, hidden)
		self.layer_2 = nn.Linear(hidden, hidden)
		self.layer_3 = nn.Linear(hidden, hidden)
		self.layer_4 = nn.Linear(hidden, hidden)
		self.layer_5 = nn.Linear(hidden, hidden)
		self.layer_6 = nn.Linear(hidden, out_features)
		self.relu = nn.ReLU()
		self.tanh = nn.Tanh()

	def forward(self, x):
		x = self.relu(self.layer_1(x))
		x = self.tanh(self.layer_2(x))
		x = self.relu(self.layer_3(x))
		x = self.tanh(self.layer_4(x))
		x = self.relu(self.layer_5(x))
		return self.layer_6(x)

# Instantiate the model
## Your code here ##
moon_model = MoonModelV0(in_features=2, hidden=32, out_features=1).to(device)


## 3. Configurar una función de pérdida compatible con clasificación binaria y un optimizador para usar al entrenar el modelo integrado en 2.

In [ ]:
# Setup loss function
loss_fn = nn.BCEWithLogitsLoss()

# Setup optimizer to optimize model's parameters
optimizer = torch.optim.SGD(moon_model.parameters(), lr=0.1)

## 4. Cree un bucle de entrenamiento y prueba para ajustar el modelo que creó en 2 a los datos que creó en 1.
*  Realice un análisis de avance del modelo para ver los resultados en forma de logits, probabilidades de predicción y etiquetas.
* Para medir la precisión del modelo, puede crear su propia función de precisión o usar la función de precisión de [TorchMetrics](https://torchmetrics.readthedocs.io/en/latest/).
* Entrene el modelo durante el tiempo suficiente para que alcance un accuracy superior al 96.
* El bucle de entrenamiento debe mostrar el progreso cada 10 épocas de la pérdida y la precisión de los conjuntos de entrenamiento y prueba del modelo.


In [ ]:
#¿Qué sale de nuestro modelo?

# logits
print("Logits: Es el valor crudo que nos entrega nuestro modelo al predecir un input, este aun no corresponde a una probabilidad hasta que se le haya sido aplicada la funcion de activacion sigmoide")

# Predicción probabilidades
print("Pred probs: Corresponde a la distribución de probabilidades que predice nuestro modelo respecto a las distintas etiquetas en nuestro problema, en este caso en particular 2.")

# Predicción etiquetas
print("Pred etiquetas: Correspondera a la etiqueta a la cual nuestro modelo predice que pertenece un determinado input.")


In [ ]:
!pip -q install torchmetrics
from torchmetrics import Accuracy

## TODO: Descomente este código para utilizar la función Accuracy
acc_fn = Accuracy(task="multiclass", num_classes=2).to(device)

In [ ]:
## TODO: Descomente esto para establecer la semilla

torch.manual_seed(RANDOM_SEED)

# Configure epochs
epochs = 1000

# Enviar datos al dispositivo
X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Recorrer los datos
for epoch in range(epochs):
	### Entrenamiento

	# 1. Forward pass
	y_logits = moon_model(X_train).squeeze()

	# Convierte los logits en probabilidades de predicción
	y_prob = torch.sigmoid(y_logits)

	# Convierte las probabilidades de predicción en etiquetas de predicción
	y_pred = torch.round(y_prob)

	# 2. Calcule la loss
	train_loss = loss_fn(y_logits, y_train)

	# Calcule  accuracy
	train_acc = acc_fn(y_pred, y_train.int())

	# 3.  gradients
	optimizer.zero_grad()

	# 4. Loss backward (backpropagation)
	train_loss.backward()

	# 5. optimizer (gradient descent)
	optimizer.step()

	### Testing
	moon_model.eval()
	with torch.inference_mode():
		# 1. Forward pass (to get the logits)
		test_logits = moon_model(X_test).squeeze()

		# Convierte las probabilidades de predicción en etiquetas de predicción
		test_prob = torch.sigmoid(test_logits)
		test_pred = torch.round(test_prob)

		# 2. Cacular el test loss/acc
		test_loss = loss_fn(test_logits, y_test)
		test_acc = acc_fn(test_pred, y_test.int())

		# Print lo que sucede cada 100 epochs
		if epoch % 100 == 0:
			print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f}, Train Accuracy: {train_acc*100:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc*100:.2f}%")

## 5. Make predictions with your trained model and plot them using the `plot_decision_boundary()` function created in this notebook.

In [ ]:
# Plot the model predictions
import numpy as np

def plot_decision_boundary(model, X, y):


	model.to("cpu")
	X, y = X.to("cpu"), y.to("cpu")

	# Source - https://madewithml.com/courses/foundations/neural-networks/
	# (with modifications)
	x_min, x_max = X[:, 0].min() - 0.1, X[:, 0].max() + 0.1
	y_min, y_max = X[:, 1].min() - 0.1, X[:, 1].max() + 0.1
	xx, yy = np.meshgrid(np.linspace(x_min, x_max, 101),
						 np.linspace(y_min, y_max, 101))

	# Make features
	X_to_pred_on = torch.from_numpy(np.column_stack((xx.ravel(), yy.ravel()))).float()

	# Make predictions
	model.eval()
	with torch.inference_mode():
		y_logits = model(X_to_pred_on)

	# Test for multi-class or binary and adjust logits to prediction labels
	if len(torch.unique(y)) > 2:
		y_pred = torch.softmax(y_logits, dim=1).argmax(dim=1) # mutli-class
	else:
		y_pred = torch.round(torch.sigmoid(y_logits)) # binary

	# Reshape preds and plot
	y_pred = y_pred.reshape(xx.shape).detach().numpy()
	plt.contourf(xx, yy, y_pred, cmap=plt.cm.RdYlBu, alpha=0.7)
	plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
	plt.xlim(xx.min(), xx.max())
	plt.ylim(yy.min(), yy.max())

In [ ]:
# Plot decision boundaries for training and test sets
plot_decision_boundary(moon_model, X_train, y_train)

In [ ]:
plot_decision_boundary(moon_model, X_test, y_test)

## 6. Cree un conjunto de datos de múltiples clases utilizando la función [spirals data creation function from CS231n](https://cs231n.github.io/neural-networks-case-study/) (see below for the code).
* SDividir los datos en conjuntos de entrenamiento y prueba (80 % entrenamiento, 20 % prueba) y convertirlos en tensores de PyTorch.
* Construya un modelo capaz de ajustar los datos (es posible que necesite una combinación de capas lineales y no lineales).
* Construya una función de pérdida y un optimizador capaz de manejar datos de múltiples clases (extensión opcional: use el optimizador Adam en lugar de SGD, es posible que tenga que experimentar con diferentes valores de la tasa de aprendizaje para que funcione).
* Realice un ciclo de entrenamiento y prueba para los datos de múltiples clases y entrene un modelo en él para alcanzar una precisión de prueba de más del 95 % (puede usar cualquier función de medición de precisión que desee aquí): 1000 épocas deberían ser suficientes.
* Grafique los límites de decisión en el conjunto de datos de espirales a partir de las predicciones de su modelo; la función `plot_decision_boundary()` también debería funcionar para este conjunto de datos.


In [ ]:
# Code for creating a spiral dataset from CS231n
import numpy as np
import matplotlib.pyplot as plt
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
N = 100 # number of points per class
D = 2 # dimensionality
K = 3 # number of classes
X = np.zeros((N*K,D)) # data matrix (each row = single example)
y = np.zeros(N*K, dtype='uint8') # class labels
for j in range(K):
	ix = range(N*j,N*(j+1))
	r = np.linspace(0.0,1,N) # radius
	t = np.linspace(j*4,(j+1)*4,N) + np.random.randn(N)*0.2 # theta
	X[ix] = np.c_[r*np.sin(t), r*np.cos(t)]
	y[ix] = j
# lets visualize the data
plt.scatter(X[:, 0], X[:, 1], c=y, s=40, cmap=plt.cm.RdYlBu)
plt.show()

In [ ]:
import torch
X = torch.from_numpy(X).type(torch.float)
y = torch.from_numpy(y).type(torch.LongTensor)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)
len(X_train), len(X_test), len(y_train), len(y_test)


In [ ]:

!pip -q install torchmetrics
from torchmetrics import Accuracy

## TODO: uncomment the two lines below to send the accuracy function to the device
acc_fn = Accuracy(task="multiclass", num_classes=3).to(device)

In [ ]:
#
device = "cuda" if torch.cuda.is_available() else "cpu"

# Create model by subclassing nn.Module
class SpiralModelV0(nn.Module):
	## Your code here ##
	def __init__(self, in_features=2, hidden=64, out_features=3):
		super().__init__()
		self.layer_1 = nn.Linear(in_features, hidden)
		self.layer_2 = nn.Linear(hidden, hidden)
		self.layer_3 = nn.Linear(hidden, hidden)
		self.layer_4 = nn.Linear(hidden, out_features)
		self.relu = nn.ReLU()

	def forward(self, x):
		x = self.relu(self.layer_1(x))
		x = self.relu(self.layer_2(x))
		x = self.relu(self.layer_3(x))
		return self.layer_4(x)

# Instantiate model and send it to device
spiral_model = SpiralModelV0(in_features=2, hidden=64, out_features=3).to(device)

In [ ]:
# Setup data to be device agnostic

# Print out first 10 untrained model outputs (forward pass)
print("Logits:")
## Your code here ##
untrained_preds = spiral_model(X_test.to(device))
print(f"First 10 predictions logits:\n{untrained_preds[:10]}")

print("\nPred probs:")
## Your code here ##
untrained_probs = torch.sigmoid(test_logits)
print(f"First 10 predictions probabilities:\n{untrained_probs[:10]}")

print("\nPred labels:")
## Your code here ##
untrained_labels = torch.round(test_prob)
print(f"First 10 predictions labels:\n{untrained_labels[:10]}")

In [ ]:
# Setup loss function and optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=spiral_model.parameters(), lr=1e-3)

In [ ]:
# Build a training loop for the model
torch.manual_seed(RANDOM_SEED)
epochs = 1000

X_train, y_train = X_train.to(device), y_train.to(device)
X_test, y_test = X_test.to(device), y_test.to(device)

# Loop over data
for epoch in range(epochs):
    ## Training

	# 1. Forward pass
	train_logits = spiral_model(X_train)

	# 2. Calcula la loss
	train_loss = loss_fn(train_logits, y_train)
	train_acc = acc_fn(train_logits, y_train)

	# 3. Optimizador zero grad
	optimizer.zero_grad()

	# 4. Loss backward
	train_loss.backward()

	# 5. Optimizer step
	optimizer.step()

	## Testing
	spiral_model.eval()
	with torch.inference_mode():
		# 1. Forward pass
		test_logits = spiral_model(X_test)

		# 2. Caculate loss and acc
		test_loss = loss_fn(test_logits, y_test)
		test_acc = acc_fn(test_logits, y_test)

		# Print out what's happening every 100 epochs
		if epoch % 100 == 0:
			print(f"Epoch: {epoch} | Train Loss: {train_loss:.5f}, Train Accuracy: {train_acc*100:.2f}% | Test Loss: {test_loss:.5f}, Test Accuracy: {test_acc*100:.2f}%")

In [ ]:
# Plot decision boundaries for training and test sets
plot_decision_boundary(spiral_model, X_train, y_train)

In [ ]:
# Plot decision boundaries for training and test sets
plot_decision_boundary(spiral_model, X_test, y_test)